In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import h5py

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
from sklearn.preprocessing import StandardScaler
standScaler = StandardScaler()

x_train_scaler = standScaler.fit_transform(x_train.reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaler = standScaler.transform(x_test.reshape(-1, 1)).reshape(-1, 28, 28)

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [6]:
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [7]:
history = model.fit(x_train_scaler, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4700 - accuracy: 0.8275 - val_loss: 0.3930 - val_accuracy: 0.8528
Epoch 2/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3571 - accuracy: 0.8689 - val_loss: 0.3703 - val_accuracy: 0.8651
Epoch 3/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3223 - accuracy: 0.8799 - val_loss: 0.3489 - val_accuracy: 0.8730
Epoch 4/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2970 - accuracy: 0.8890 - val_loss: 0.3582 - val_accuracy: 0.8704
Epoch 5/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2780 - accuracy: 0.8944 - val_loss: 0.3300 - val_accuracy: 0.8817
Epoch 6/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2605 - accuracy: 0.9026 - val_loss: 0.3381 - val_accuracy: 0.8835
Epoch 7/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2452 - accuracy: 0.9063 - val_loss: 0.3252 - val_accuracy:

In [8]:
model.evaluate(x_test_scaler, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3553 - accuracy: 0.8814


[0.35529741644859314, 0.8813999891281128]

### 保存模型

In [17]:
tf.saved_model.save(model, "./keras_fashin_mnist")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./keras_fashin_mnist\assets


In [19]:
loaded_model = tf.saved_model.load("./keras_fashin_mnist")

In [25]:
!saved_model_cli show --dir ./keras_fashin_mnist --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          flatten_i

In [28]:
!saved_model_cli show --dir ./keras_fashin_mnist --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_2'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [35]:
print(loaded_model.signatures.keys())

KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(flatten_input) at 0x2335407DF40>}))


In [36]:
inference = loaded_model.signatures['serving_default']